In [6]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd


import matplotlib
from matplotlib import pyplot as plt
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier



from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV, StratifiedKFold, \
    cross_validate, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, \
    mean_absolute_error, make_scorer, brier_score_loss, roc_curve

from sklearn.preprocessing import OneHotEncoder

from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from tensorflow import keras

import math
import sys

import statsmodels.api as sm
import scipy.stats as stats



import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"s

matplotlib.use('Agg')



__author__ = "Dor Ma'ayan"
__email__ = "grano@ifi.uzh.ch"
__license__ = "MIT"


CSV_PATH = "complete-frame.csv"
CSV_MINER_PATH = "testminereffectiveness.csv"
DATA_DIR = "results"


def label_rename1 (row):
    return row['path_test'].split('/')[len(row['path_test'].split('/')) - 1].split('.')[0]

def label_rename2 (row):
    return row['path_src'].split('/')[len(row['path_src'].split('/')) - 1].split('.')[0]

def load_quartile(frame):
    low, high = frame.mutation.quantile([0.25,0.75])
    frame_low = frame.query('mutation<{low}'.format(low=low))
    frame_high = frame.query('mutation>{high}'.format(high=high))
    frame_low['mutation'] = 0
    frame_high['mutation'] = 1
    frame = pd.concat([frame_low, frame_high], ignore_index=True)
    frame = frame.sample(frac=1).reset_index(drop=True)
    return frame;

def load_frame():
    
    d = {'TestClassName' : 'ClassName',
         'Vocabulary' : 'Vocabulary_prod',
         'Word' : 'Word_prod', 
         'Non Whithe Characters' : 'Non Whithe Characters_prod',
         'No. Methods' : 'No. Methods_prod',
         'Special' : 'Special_prod',
     'No. Method Invoctions' : 'No. Method Invoctions_prod',
    'AST size' : 'AST size_prod', 'Max Depth' : 'Max Depth_prod',
         'Deg2' : 'Deg2_prod',
         'DegPerm' : 'DegPerm_prod',
         'No. Break' : 'No. Break_prod',
         'No. Continue' : 'No. Continue_prod',
     'Avg Depth' : 'Avg Depth_prod', 'Dexterity' : 'Dexterity_prod',
    'No. Expressions' : 'No. Expressions_prod', 'No. Try' : 'No. Try_prod', 'No. Catch' : 'No. Catch_prod',
     'No. Loop' : 'No. Loop_prod', 'No. Conditions' : 'No. Conditions_prod', 'No. Else' : 'No. Else_prod'}
    
    
    frame1 = pd.read_csv(CSV_PATH, sep=",")
    frame1 = frame1.sample(frac=1).reset_index(drop=True)
    frame1['TestClassName'] = frame1.apply(lambda row: label_rename1(row), axis=1)
    frame1['ClassName'] = frame1.apply(lambda row: label_rename2(row), axis=1)
        
    
    frame2 = pd.read_csv(CSV_MINER_PATH, sep=',')
    
    frame3 = pd.read_csv(CSV_MINER_PATH, sep=',')
    frame3 = frame3.rename(columns = d)
    frame3 = frame3.drop(['Bad API', 'Junit', 'Hamcrest', 'Mockito', 'Nº','Project'], axis=1)
    
    
    frame = pd.merge(frame1, frame2, on='TestClassName')
    
    frame = pd.merge(frame, frame3, on='ClassName')
    

    frame = frame.drop(['project', 'module', 'path_test','test_name','path_src',
                        'commit', 'class_name'], axis=1)
    frame = frame.sample(frac=1).reset_index(drop=True)
    frame = frame.dropna()

    return frame


def load_quartile(frame):
    low, high = frame.mutation.quantile([0.25,0.75])
    frame_low = frame.query('mutation<{low}'.format(low=low))
    frame_high = frame.query('mutation>{high}'.format(high=high))
    frame_low['mutation'] = 0
    frame_high['mutation'] = 1
    frame = pd.concat([frame_low, frame_high], ignore_index=True)
    frame = frame.sample(frac=1).reset_index(drop=True)
    return frame;



def load_all_data(frame):
    columns = ['isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting', 'LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod', 'LOC_test',
       'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test',
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'test_readability', 'No. Methods', 'Vocabulary', 'Word',
               'Special', 'Non Whithe Characters', 'No. Method Invoctions', 'AST size', 'Max Depth',
               'Avg Depth', 'Deg2', 'DegPerm', 'Dexterity', 'No. Expressions', 'No. Try', 'No. Catch',
               'No. Loop', 'No. Break', 'No. Continue', 'No. Conditions', 'No. Else', 'Bad API',
               'Junit', 'Hamcrest', 'Mockito', 'No. Methods_prod', 'Vocabulary_prod', 'Word_prod',
               'Special_prod', 'Non Whithe Characters_prod', 'No. Method Invoctions_prod', 'AST size_prod',
               'Max Depth_prod', 'Avg Depth_prod', 'Deg2_prod', 'DegPerm_prod', 'Dexterity_prod',
               'No. Expressions_prod', 'No. Try_prod', 'No. Catch_prod', 'No. Loop_prod', 'No. Break_prod',
               'No. Continue_prod', 'No. Conditions_prod', 'No. Else_prod']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

def load_all_data_dynamic(frame):
    columns = ['line_coverage', 'isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting', 'LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod', 'LOC_test',
       'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test',
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'test_readability', 'No. Methods', 'Vocabulary', 'Word',
               'Special', 'Non Whithe Characters', 'No. Method Invoctions', 'AST size', 'Max Depth',
               'Avg Depth', 'Deg2', 'DegPerm', 'Dexterity', 'No. Expressions', 'No. Try', 'No. Catch',
               'No. Loop', 'No. Break', 'No. Continue', 'No. Conditions', 'No. Else', 'Bad API',
               'Junit', 'Hamcrest', 'Mockito', 'No. Methods_prod', 'Vocabulary_prod', 'Word_prod',
               'Special_prod', 'Non Whithe Characters_prod', 'No. Method Invoctions_prod', 'AST size_prod',
               'Max Depth_prod', 'Avg Depth_prod', 'Deg2_prod', 'DegPerm_prod', 'Dexterity_prod',
               'No. Expressions_prod', 'No. Try_prod', 'No. Catch_prod', 'No. Loop_prod', 'No. Break_prod',
               'No. Continue_prod', 'No. Conditions_prod', 'No. Else_prod']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_all_their_data(frame):
    columns = ['line_coverage', 'isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting', 'LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod', 'LOC_test',
       'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test',
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'test_readability']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

def load_all_their_test_data(frame):
    columns = ['isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting','LOC_test',
       'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test', 'test_readability']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

def load_all_test_data(frame):
    columns = ['isAssertionRoulette',
       'isEagerTest', 'isLazyTest', 'isMysteryGuest',
       'isSensitiveEquality', 'isResourceOptimism', 'isForTestersOnly',
       'isIndirectTesting', 'LOC_test',
       'HALSTEAD_test', 'RFC_test', 'CBO_test', 'MPC_test', 'IFC_test',
       'DAC_test', 'DAC2_test', 'LCOM1_test', 'LCOM2_test', 'LCOM3_test',
       'LCOM4_test', 'CONNECTIVITY_test', 'LCOM5_test', 'COH_test',
       'TCC_test', 'LCC_test', 'ICH_test', 'WMC_test', 'NOA_test',
       'NOPA_test', 'NOP_test', 'McCABE_test', 'BUSWEIMER_test',
       'test_readability', 'No. Methods', 'Vocabulary', 'Word',
               'Special', 'Non Whithe Characters', 'No. Method Invoctions', 'AST size', 'Max Depth',
               'Avg Depth', 'Deg2', 'DegPerm', 'Dexterity', 'No. Expressions', 'No. Try', 'No. Catch',
               'No. Loop', 'No. Break', 'No. Continue', 'No. Conditions', 'No. Else', 'Bad API',
               'Junit', 'Hamcrest', 'Mockito']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_all_production_data(frame):
    columns = ['LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod',
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'No. Methods_prod', 'Vocabulary_prod', 'Word_prod',
               'Special_prod', 'Non Whithe Characters_prod', 'No. Method Invoctions_prod', 'AST size_prod',
               'Max Depth_prod', 'Avg Depth_prod', 'Deg2_prod', 'DegPerm_prod', 'Dexterity_prod',
               'No. Expressions_prod', 'No. Try_prod', 'No. Catch_prod', 'No. Loop_prod', 'No. Break_prod',
               'No. Continue_prod', 'No. Conditions_prod', 'No. Else_prod']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)


def load_all_production_data_line_coverage(frame):
    columns = ['LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod',
       'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability', 'No. Methods_prod', 'Vocabulary_prod', 'Word_prod',
               'Special_prod', 'Non Whithe Characters_prod', 'No. Method Invoctions_prod', 'AST size_prod',
               'Max Depth_prod', 'Avg Depth_prod', 'Deg2_prod', 'DegPerm_prod', 'Dexterity_prod',
               'No. Expressions_prod', 'No. Try_prod', 'No. Catch_prod', 'No. Loop_prod', 'No. Break_prod',
               'No. Continue_prod', 'No. Conditions_prod', 'No. Else_prod']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.line_coverage], axis = 1)
    return data_x, data_y, len(columns)


def load_all_their_production_data(frame):
    columns = ['LOC_prod', 'HALSTEAD_prod', 'RFC_prod',
       'CBO_prod', 'MPC_prod', 'IFC_prod', 'DAC_prod', 'DAC2_prod',
       'LCOM1_prod', 'LCOM2_prod', 'LCOM3_prod', 'LCOM4_prod',
       'CONNECTIVITY_prod', 'LCOM5_prod', 'COH_prod', 'TCC_prod',
       'LCC_prod', 'ICH_prod', 'WMC_prod', 'NOA_prod', 'NOPA_prod',
       'NOP_prod', 'McCABE_prod', 'BUSWEIMER_prod', 'csm_CDSBP', 'csm_CC', 'csm_FD', 'csm_Blob', 'csm_SC', 'csm_MC',
       'csm_LM', 'csm_FE', 'prod_readability']

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

    data_x = frame[columns].round(2)
    data_y = pd.concat([frame.mutation], axis = 1)
    return data_x, data_y, len(columns)

def get_scoring():
    """Returns the scores to evaluate the model"""
    return dict(accuracy=make_scorer(accuracy_score),
                precision=make_scorer(precision_score),
                recall=make_scorer(recall_score),
                f1_score=make_scorer(f1_score),
                roc_auc_scorer=make_scorer(roc_auc_score),
                mean_absolute_error=make_scorer(mean_absolute_error),
                brier_score=make_scorer(brier_score_loss))


frame = load_frame()
data_x, data_y, number_of_features = load_all_data(frame)
data_y = pd.concat([frame.mutation], axis = 1).round(2).values
scaler = StandardScaler()
scaler.fit(data_x)
data_x = scaler.transform(data_x)

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.10)

print(x_train.shape)

model = keras.Sequential()
model.add(keras.layers.Dense(number_of_features, activation='relu', input_dim=number_of_features))
model.add(keras.layers.Dense(40, activation='relu'))
model.add(keras.layers.Dense(20, activation='relu'))
model.add(keras.layers.Dense(1))

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae','mse'])

history = model.fit(x_train, y_train, epochs=400, verbose=1) #, callbacks=[early_stopping_monitor])


test_loss, test_mae, test_mse = model.evaluate(x_test, y_test)

print('MAE: {}'.format(test_mae))
y_pred = model.predict(x_test)
y_pred = np.concatenate(y_pred).tolist()
y_testi = np.concatenate(y_test).tolist()
plt.scatter(*zip(*list(zip(y_testi,y_pred))))
plt.ylabel('Predicted')
plt.xlabel('Real')
plt.show()
plt.savefig("foo.pdf", bbox_inches='tight')
plt.close()
tau, p_value = stats.pearsonr(y_pred, y_testi) #stats.kendalltau(y_pred, y_testi)


(2412, 110)
Train on 2412 samples
Epoch 1/400
2412/2412 [==============================] - 1s 424us/sample - loss: 0.1486 - mae: 0.2809 - mse: 0.1486
Epoch 2/400
2412/2412 [==============================] - 0s 88us/sample - loss: 0.0553 - mae: 0.1802 - mse: 0.0553
Epoch 3/400
2412/2412 [==============================] - 0s 88us/sample - loss: 0.0406 - mae: 0.1557 - mse: 0.0406
Epoch 4/400
2412/2412 [==============================] - 0s 93us/sample - loss: 0.0334 - mae: 0.1408 - mse: 0.0334
Epoch 5/400
2412/2412 [==============================] - 0s 92us/sample - loss: 0.0292 - mae: 0.1308 - mse: 0.0292
Epoch 6/400
2412/2412 [==============================] - 0s 97us/sample - loss: 0.0303 - mae: 0.1255 - mse: 0.0303
Epoch 7/400
2412/2412 [==============================] - 0s 94us/sample - loss: 0.0243 - mae: 0.1173 - mse: 0.0243
Epoch 8/400
2412/2412 [==============================] - 0s 90us/sample - loss: 0.0227 - mae: 0.1140 - mse: 0.0227
Epoch 9/400
2412/2412 [======================

2412/2412 [==============================] - 0s 103us/sample - loss: 0.0015 - mae: 0.0292 - mse: 0.0015
Epoch 72/400
2412/2412 [==============================] - 0s 97us/sample - loss: 0.0018 - mae: 0.0313 - mse: 0.0018
Epoch 73/400
2412/2412 [==============================] - 0s 96us/sample - loss: 0.0023 - mae: 0.0347 - mse: 0.0023
Epoch 74/400
2412/2412 [==============================] - 0s 96us/sample - loss: 0.0019 - mae: 0.0318 - mse: 0.0019
Epoch 75/400
2412/2412 [==============================] - 0s 98us/sample - loss: 0.0020 - mae: 0.0328 - mse: 0.0020
Epoch 76/400
2412/2412 [==============================] - 0s 108us/sample - loss: 0.0021 - mae: 0.0338 - mse: 0.0021
Epoch 77/400
2412/2412 [==============================] - 0s 103us/sample - loss: 0.0022 - mae: 0.0349 - mse: 0.0022
Epoch 78/400
2412/2412 [==============================] - 0s 96us/sample - loss: 0.0020 - mae: 0.0337 - mse: 0.0020
Epoch 79/400
2412/2412 [==============================] - 0s 97us/sample - loss: 0

Epoch 140/400
2412/2412 [==============================] - 0s 99us/sample - loss: 9.3991e-04 - mae: 0.0227 - mse: 9.3991e-04
Epoch 141/400
2412/2412 [==============================] - 0s 100us/sample - loss: 0.0013 - mae: 0.0266 - mse: 0.0013
Epoch 142/400
2412/2412 [==============================] - 0s 99us/sample - loss: 0.0015 - mae: 0.0292 - mse: 0.0015
Epoch 143/400
2412/2412 [==============================] - 0s 103us/sample - loss: 0.0016 - mae: 0.0291 - mse: 0.0016
Epoch 144/400
2412/2412 [==============================] - 0s 108us/sample - loss: 0.0014 - mae: 0.0275 - mse: 0.0014
Epoch 145/400
2412/2412 [==============================] - 0s 106us/sample - loss: 0.0015 - mae: 0.0284 - mse: 0.0015
Epoch 146/400
2412/2412 [==============================] - 0s 100us/sample - loss: 0.0016 - mae: 0.0292 - mse: 0.0016
Epoch 147/400
2412/2412 [==============================] - 0s 99us/sample - loss: 0.0013 - mae: 0.0262 - mse: 0.0013
Epoch 148/400
2412/2412 [==========================

2412/2412 [==============================] - 0s 103us/sample - loss: 7.1722e-04 - mae: 0.0197 - mse: 7.1722e-04
Epoch 208/400
2412/2412 [==============================] - 0s 103us/sample - loss: 7.5451e-04 - mae: 0.0202 - mse: 7.5451e-04
Epoch 209/400
2412/2412 [==============================] - 0s 103us/sample - loss: 7.2571e-04 - mae: 0.0199 - mse: 7.2571e-04
Epoch 210/400
2412/2412 [==============================] - 0s 105us/sample - loss: 8.7793e-04 - mae: 0.0218 - mse: 8.7793e-04
Epoch 211/400
2412/2412 [==============================] - 0s 115us/sample - loss: 0.0012 - mae: 0.0247 - mse: 0.0012
Epoch 212/400
2412/2412 [==============================] - 0s 107us/sample - loss: 8.6774e-04 - mae: 0.0222 - mse: 8.6774e-04
Epoch 213/400
2412/2412 [==============================] - 0s 104us/sample - loss: 8.3766e-04 - mae: 0.0219 - mse: 8.3766e-04
Epoch 214/400
2412/2412 [==============================] - 0s 103us/sample - loss: 9.6632e-04 - mae: 0.0233 - mse: 9.6632e-04
Epoch 215/400


2412/2412 [==============================] - 0s 112us/sample - loss: 9.5231e-04 - mae: 0.0232 - mse: 9.5231e-04
Epoch 273/400
2412/2412 [==============================] - 0s 103us/sample - loss: 6.2498e-04 - mae: 0.0187 - mse: 6.2498e-04
Epoch 274/400
2412/2412 [==============================] - 0s 105us/sample - loss: 6.1806e-04 - mae: 0.0186 - mse: 6.1806e-04
Epoch 275/400
2412/2412 [==============================] - 0s 103us/sample - loss: 6.2133e-04 - mae: 0.0183 - mse: 6.2133e-04
Epoch 276/400
2412/2412 [==============================] - 0s 106us/sample - loss: 6.3469e-04 - mae: 0.0173 - mse: 6.3469e-04
Epoch 277/400
2412/2412 [==============================] - 0s 110us/sample - loss: 0.0018 - mae: 0.0240 - mse: 0.0018
Epoch 278/400
2412/2412 [==============================] - 0s 105us/sample - loss: 0.0013 - mae: 0.0259 - mse: 0.0013
Epoch 279/400
2412/2412 [==============================] - 0s 100us/sample - loss: 0.0011 - mae: 0.0242 - mse: 0.0011
Epoch 280/400
2412/2412 [=====

2412/2412 [==============================] - 0s 100us/sample - loss: 6.9506e-04 - mae: 0.0192 - mse: 6.9506e-04
Epoch 338/400
2412/2412 [==============================] - 0s 101us/sample - loss: 7.3077e-04 - mae: 0.0204 - mse: 7.3077e-04
Epoch 339/400
2412/2412 [==============================] - 0s 109us/sample - loss: 6.3879e-04 - mae: 0.0189 - mse: 6.3879e-04
Epoch 340/400
2412/2412 [==============================] - 0s 114us/sample - loss: 5.7149e-04 - mae: 0.0178 - mse: 5.7149e-04
Epoch 341/400
2412/2412 [==============================] - 0s 110us/sample - loss: 5.1349e-04 - mae: 0.0167 - mse: 5.1349e-04
Epoch 342/400
2412/2412 [==============================] - 0s 101us/sample - loss: 4.0370e-04 - mae: 0.0148 - mse: 4.0370e-04
Epoch 343/400
2412/2412 [==============================] - 0s 100us/sample - loss: 3.8020e-04 - mae: 0.0147 - mse: 3.8020e-04
Epoch 344/400
2412/2412 [==============================] - 0s 100us/sample - loss: 3.9190e-04 - mae: 0.0144 - mse: 3.9190e-04
Epoch 

MAE: 0.12246084213256836


In [7]:
l = zip(list(zip(y_testi,y_pred)))


In [8]:
y_testi

[0.88,
 0.68,
 0.28,
 0.8,
 0.63,
 0.53,
 0.6,
 0.98,
 0.89,
 0.91,
 0.89,
 0.85,
 0.5,
 1.0,
 0.86,
 0.8,
 0.99,
 0.79,
 1.0,
 0.59,
 0.5,
 0.53,
 0.67,
 1.0,
 0.05,
 0.33,
 1.0,
 0.5,
 0.92,
 0.32,
 0.88,
 0.86,
 0.05,
 0.75,
 0.62,
 0.59,
 0.58,
 0.29,
 0.0,
 0.44,
 0.65,
 0.8,
 0.49,
 1.0,
 1.0,
 0.03,
 0.1,
 1.0,
 0.77,
 0.69,
 0.95,
 0.51,
 0.7,
 0.98,
 0.89,
 0.77,
 0.22,
 0.97,
 0.74,
 0.85,
 0.93,
 1.0,
 0.76,
 0.49,
 0.81,
 0.7,
 0.83,
 0.72,
 0.49,
 0.81,
 0.76,
 0.17,
 0.75,
 0.22,
 0.58,
 0.15,
 0.45,
 0.84,
 0.86,
 1.0,
 0.48,
 0.83,
 0.66,
 0.76,
 0.19,
 0.6,
 0.37,
 0.92,
 0.7,
 0.55,
 0.52,
 0.48,
 0.46,
 0.86,
 0.77,
 0.01,
 0.52,
 1.0,
 0.39,
 0.45,
 0.71,
 0.69,
 0.94,
 0.86,
 1.0,
 0.84,
 0.23,
 0.64,
 0.78,
 0.74,
 1.0,
 0.42,
 0.78,
 0.49,
 0.42,
 0.63,
 0.38,
 0.53,
 0.5,
 0.78,
 1.0,
 0.09,
 0.15,
 0.67,
 0.66,
 0.23,
 0.8,
 0.11,
 0.93,
 0.52,
 0.88,
 0.55,
 1.0,
 0.54,
 0.47,
 0.22,
 0.62,
 0.51,
 0.66,
 0.45,
 0.87,
 0.86,
 0.04,
 0.57,
 0.22,
 0.73,
 1.0,
 

In [9]:
y_pred

[0.701339602470398,
 0.6811660528182983,
 0.5560958385467529,
 0.7347605228424072,
 0.7715393304824829,
 0.600234866142273,
 0.6574651002883911,
 0.8678652048110962,
 0.9642412662506104,
 0.8547234535217285,
 0.8014798164367676,
 0.8300817012786865,
 0.5179041624069214,
 0.7771919965744019,
 0.9022994041442871,
 0.8022595643997192,
 0.7298210859298706,
 0.7195295095443726,
 1.015722393989563,
 0.39811691641807556,
 0.49374255537986755,
 0.5887069702148438,
 0.3722723126411438,
 0.8083269596099854,
 0.12761826813220978,
 0.3251449167728424,
 1.0053842067718506,
 0.6612609624862671,
 0.5741885304450989,
 0.48485586047172546,
 0.6661880016326904,
 0.7198034524917603,
 0.07557301223278046,
 0.5436270236968994,
 0.7708116769790649,
 0.5545024275779724,
 0.3595973253250122,
 0.3234959542751312,
 0.20701543986797333,
 0.5786458253860474,
 0.7028197050094604,
 0.9229729175567627,
 0.7111343145370483,
 1.0072064399719238,
 0.7823516130447388,
 0.10935890674591064,
 0.11395472288131714,
 0.97004

In [10]:
import csv 

rows = zip(y_testi,y_pred)
with open('returns-static.csv', 'w') as f:
    writer = csv.writer(f)
    for row in rows:
        writer.writerow(row)